In [2]:
import os
import csv
import time
import pandas
import librosa
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

In [3]:
"""
    Description: Function for getting all recording sessions in a folder
    @param [Array] y: The audio signal
    @param [Integer] cutoff: The cutoff or limit of frequency
    @param [String] btype: Type of filter, highpass or lowpass ['high', 'low']
    @param [Integer] sr: Sample rate of signal
    Return: [Array] The new filtered signal without the frequencies below or above the cutoff depending on butterworth-type
""" 

def butter_filter(y, cutoff, btype, sr):
    order = 9
    nyq = 0.5 * sr
    normal_cutoff = cutoff / nyq
    # Getting the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype=btype, analog=False)
    y = signal.filtfilt(b, a, y)
    return y

"""
    Description: Example function for applying Fast-Fourier Transform on a signal
    @param [Array] y: The audio signal
    @param [Integer] sr: The sample rate of the signal
    @param [Boolean] plot: 'True' or 'False' for wanting to show the fft-plot
    @param [Boolean] norm: 'True' or 'False' for wanting to normalize the signal
    Return: [Array, Array] The frequencies and belonging magnitudes from the signal
""" 

def apply_fft(y, sr, plot, norm):
    if norm == True:
        yf = np.fft.fft(y, norm='ortho')
        norm = " - Normalized"
    else: 
        yf = np.fft.fft(y, norm=None)
        norm = " "
    magnitude = np.abs(yf.real)
    frequency = np.linspace(0, sr, len(yf))
    if plot == True:
        plt.figure(figsize=(10,5))
        plt.plot(frequency[:int(len(frequency)/2)], magnitude[:int(len(magnitude)/2)])
        plt.xlabel("Frequency [Hz]")
        plt.ylabel("Magnitude [dB]")
        plt.title("Frequency-Magnitude (FFT) Spectrum" + norm)
        plt.show()
    return [frequency[:int(len(frequency)/2)], magnitude[:int(len(magnitude)/2)]]

"""
    Description: Function for applying Hann Windowing to a signal to ensure no artifacts
    @param [Array] y: The audio signal
    @param [Integer] sr: The sample rate of the signal
    Return: [Array] The windowed signal
"""
def apply_windowing(y, sr):
    N = len(y)
    w = signal.hann(N)
    yw = y*w
    return yw

"""
    Description: Function for creating a plot of a feature of a signal
    @param [Array] y: The audio signal
    @param [Integer] mean: The average value of the feature
    @param [String] ylabel: The label for the plot which explains which feature is shown
    Return: None (but shows the plot)
""" 

def plot_feature(y, mean, ylabel):
    times = librosa.times_like(y)
    means = []
    for i in range(len(times)):
        means.append(mean)
    plt.plot(times, y.T)
    plt.plot(times, means)
    plt.xlabel("Time [s]")
    plt.ylabel(ylabel)
    plt.show()

In [4]:
"""
    Description: The following four functions are for getting features of a signal
    @param [Array] y: The audio signal
    @param [Integer] sr: The sample rate of the signal
    @param [Integer] n_fft: The window size used for the embedded fast fourier transform
    Return: [Float] The feature value
""" 
    
def get_centroid(y, sr):
    n_fft = len(y)
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=n_fft, win_length=None, window='hann')[0][0]
    return centroid

def get_rolloff(y, sr, n_fft):
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=n_fft, win_length=None, window='hann')[0][0]
    return rolloff

def get_bandwidth(y, sr, n_fft):
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=n_fft, win_length=None, window='hann')[0][0]
    return bandwidth

def get_flatness(y, n_fft):
    flatness = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=n_fft, win_length=None, window='hann')[0][0]
    return flatness

"""
    Description: Function for getting the relative power of a signal
    @param [Array] y: The audio signal
    Return: [Float] The relative power value
""" 

def get_relative_power(y, sr):
    y_win = apply_windowing(y, sr)
    relative_power = np.max(y_win)-np.min(y_win)
    return relative_power

"""
    Description: Function for getting the power spectral density feature of a signal
    @param [Array] y: The audio signal
    Return: [Float] The rms value
""" 

def get_psd(y):
    frequency, psd = signal.welch(x=y, window='hann')
    psd = np.mean(psd)
    return psd

"""
    Description: Function for getting the largest amplitude of a signal
    @param [Array] y: The audio signal
    Return: [Float] The largest amplitude value
""" 

def get_max_amplitude(y, sr):
    y_win = apply_windowing(y, sr)
    max_amp = np.max(y_win)
    return max_amp

"""
    Description: Function for getting the root mean square feature of a signal
    @param [Array] y: The audio signal
    Return: [Float] The rms value
""" 
def get_rms(y, sr):
    y_win = apply_windowing(y, sr)
    rms = np.mean(librosa.feature.rms(y=y_win))
    return rms


In [5]:
"""
    Description: (Not used for main dataframe) Function for saving the centroid feature to a dataframe
    @param [Pandas DataFrame Object] df: The dataframe of audio signals
    Return: [Array<Float>] A list of the centroid value for each signal (also prints a plot comparing the mean centroid value)
"""     

def save_centroids(df):
    start_time = time.time()
    counter = 0
    paths = df['Path']
    list_centroids = []
    for path in paths:
        counter += 1
        y, sr = librosa.load(path, sr=None) #Endre frames-length?
        y_filtered = butter_filter(y=y, cutoff=4000, btype='high', sr=sr)
        n_fft = len(y_filtered)
        list_centroids.append(get_centroid(y=y_filtered, sr=sr))
        centroids = librosa.feature.spectral_centroid(y=y_filtered, sr=sr)
        if counter % 100 == 0 or counter == 1:
            print('Finished with file', str(counter), 'out of', str(len(paths)), '(', str(np.round(counter/len(paths)*100, 4)), '% )')
    df['Centroid'] = list_centroids
    plot_feature(centroids, np.mean(centroids), "Centroid" )
    print("Mean Centroid: ", np.mean(centroids))
    print("One-Frame Centroid:", list_centroids[0])
    print('Execution time: ', str(time.time()-start_time))
    return list_centroids


In [6]:
"""
    Description: Function for saving 9 audio features to the dataframe 
    @param [Pandas DataFrame Object] df: The dataframe of audio signals
    Return: [Array<Arrayx9>] A list of 9 feature-lists containing the audio features for all signals
"""

def save_simple_features_to_df(df):
    start_time = time.time()
    counter = 0
    paths = df['Path']
    list_rolloff = []
    list_bandwidth = []
    list_flatness = []
    for path in paths:
        counter += 1
        y, sr = librosa.load(path, sr=None) 
        y_filtered = butter_filter(y=y, cutoff=4000, btype='high', sr=sr)
        n_fft = len(y_filtered)
        list_rolloff.append(get_rolloff(y=y_filtered, sr=sr, n_fft=n_fft))
        list_bandwidth.append(get_bandwidth(y=y_filtered, sr=sr, n_fft=n_fft))
        list_flatness.append(get_flatness(y=y_filtered, n_fft=n_fft))
        if counter % 100 == 0 or counter == 1:
            print('Finished with file', str(counter), 'out of', str(len(paths)), '(', str(np.round(counter/len(paths)*100, 4)), '% )')    
    df['Bandwidth'] = list_bandwidth
    df['Rolloff'] = list_rolloff
    df['Flatness'] = list_flatness
    print('Execution time: ', str(time.time()-start_time))
    return [list_rolloff, list_bandwidth, list_flatness]

In [ ]:
"""
    Description: Function for saving the psd feature to the dataframe
    @param [Pandas DataFrame Object] df: The dataframe of audio signals
    Return: [Array<Float>] A list of the psd value for each signal 
"""  

def save_high_frequency_features(df):
    start_time = time.time()
    counter = 0
    paths = df['Path']
    list_psd = []
    list_amp = []
    for path in paths:
        counter += 1
        y, sr = librosa.load(path, sr=None) 
        y_filtered = butter_filter(y=y, cutoff=20000, btype='high', sr=sr)
        n_fft = len(y_filtered)
        list_psd.append(get_psd(y=y_filtered))
        list_amp.append(get_max_amplitude(y_filtered, sr))
        if counter % 100 == 0 or counter == 1:
            print('Finished with file', str(counter), 'out of', str(len(paths)), '(', str(np.round(counter/len(paths)*100, 4)), '% )')
    df['PSD >20kHz'] = list_psd
    df['MAX_AMP >20kHz'] = list_amp
    print('Execution time: ', str(time.time()-start_time))
    return list_psd, list_amp

In [7]:
"""
    Description: Function for normalizing features into the scale [0, 1]
    @param [Pandas DataFrame Object] df: The dataframe of audio signals
    Return: [Pandas DataFrame Object] A new dataframe with normalized values
"""
def normalize_features(df):
    for column in df.columns[3:]:
        max_feature = np.max(df[str(column)])
        min_feature = np.min(df[str(column)])
        new_values = (df[str(column)]-min_feature)/(max_feature-min_feature)
        df[str(column)] = new_values
    return df

In [ ]:
  """
    Description: Function for calculating rms and centroid of a signal over frequency-intervalls of 4000Hz
    @param [Array] y: The audio signal
    @param [Integer] sr: The sample rate of the signal
    Return: [Array<Float>] A list of rms values for all six frequency-bins
""" 

def get_bin_list(y, sr):
    y_low = butter_filter(y=y, cutoff=4000, btype='low', sr=sr)
    y_med1 = butter_filter(y=(butter_filter(y=y, cutoff=8000, btype='low', sr=sr)), cutoff=4000, btype='high', sr=sr)
    y_med2 = butter_filter(y=(butter_filter(y=y, cutoff=12000, btype='low', sr=sr)), cutoff=8000, btype='high', sr=sr)
    y_med3 = butter_filter(y=(butter_filter(y=y, cutoff=16000, btype='low', sr=sr)), cutoff=12000, btype='high', sr=sr)
    y_med4 = butter_filter(y=(butter_filter(y=y, cutoff=20000, btype='low', sr=sr)), cutoff=16000, btype='high', sr=sr)
    y_high = butter_filter(y=y, cutoff=20000, btype='high', sr=sr)
    cent_low = get_centroid(y_low, sr)
    cent1 = get_centroid(y_med1, sr)
    cent2 = get_centroid(y_med2, sr)
    cent3 = get_centroid(y_med3, sr)
    cent4 = get_centroid(y_med4, sr)
    cent_high = get_centroid(y_high, sr)
    cent_list = [cent_low, cent1, cent2, cent3, cent4, cent_high]
    rms_low = get_rms(y_low, sr)
    rms1 = get_rms(y_med1, sr)
    rms2 = get_rms(y_med2, sr)
    rms3 = get_rms(y_med3, sr)
    rms4 = get_rms(y_med4, sr)
    rms_high = get_rms(y_high, sr)
    rms_list = [rms_low, rms1, rms2, rms3, rms4, rms_high]
    return cent_list, rms_list

def save_bin_features_to_df(df):
    start_time = time.time()
    counter = 0
    paths = df['Path']
    # Initializing lists to use to store values for each signal
    list_cent1, list_cent2, list_cent3, list_cent4, list_cent5, list_cent6 = [], [], [], [], [], []
    list_rms1, list_rms2, list_rms3, list_rms4, list_rms5, list_rms6 = [], [], [], [], [], []
    for path in paths:
        counter += 1
        y, sr = librosa.load(path, sr=None) 
        cent_list, rms_list = get_bin_list(y, sr)
        list_cent1.append(cent_list[0])
        list_cent2.append(cent_list[1])       
        list_cent3.append(cent_list[2])
        list_cent4.append(cent_list[3])
        list_cent5.append(cent_list[4])
        list_cent6.append(cent_list[5])
        list_rms1.append(rms_list[0])
        list_rms2.append(rms_list[1])       
        list_rms3.append(rms_list[2])
        list_rms4.append(rms_list[3])
        list_rms5.append(rms_list[4])
        list_rms6.append(rms_list[5])
        if counter % 100 == 0 or counter == 1:
            print('Finished with file', str(counter), 'out of', str(len(paths)), '(', str(np.round(counter/len(paths)*100, 4)), '% )')    
    df['Centroid 0-4kHz'] = list_cent1
    df['Centroid 4-8kHz'] = list_cent2
    df['Centroid 8-12kHz'] = list_cent3
    df['Centroid 12-16kHz'] = list_cent4
    df['Centroid 16-20kHz'] = list_cent5
    df['Centroid 20-24kHz'] = list_cent6
    df['RMS 0-4kHz'] = list_rms1
    df['RMS 4-8kHz'] = list_rms2
    df['RMS 8-12kHz'] = list_rms3
    df['RMS 12-16kHz'] = list_rms4
    df['RMS 16-20kHz'] = list_rms5
    df['RMS 20-24kHz'] = list_rms6
    list_centroids = [list_cent1, list_cent2, list_cent3, list_cent4, list_cent5, list_cent6]
    list_rms = [list_rms1, list_rms2, list_rms3, list_rms4, list_rms5, list_rms6]
    print('Execution time: ', str(time.time()-start_time))
    return list_centroids, list_rms


"""
    Adding all feature to a dataframe with 1920 files took around 2-3 hours
"""


In [8]:
"""
    This is the part of the code that starts all the functions above in the correct order.
    The result of this code snippet is two new .csv-files with features for the tube and vent signals.
"""

df_tube = pandas.read_csv('tube_files.csv')
df_vent = pandas.read_csv('vent_files.csv')

save_simple_features_to_df(df_tube)
save_bin_features_to_df(df_tube)   
save_high_frequency_features(df_tube)

df_norm = normalize_features(df_tube)
df_norm.to_csv('new_features_tube.csv', index=False)

save_simple_features_to_df(df_vent)
save_bin_features_to_df(df_vent)
save_high_frequency_features(df_vent)

df_norm2 = normalize_features(df_vent)
df_norm2.to_csv('new_features_vent.csv', index=False)